<a href="https://colab.research.google.com/github/wooookim/ML-DL/blob/main/%EC%84%9C%EC%9A%B8%EC%8B%9C_%ED%8F%89%EA%B7%A0_%EA%B8%B0%EC%98%A8_%EC%98%88%EC%B8%A1_29.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

In [6]:
data = pd.read_csv('/content/train.csv')
submission = pd.read_csv('/content/sample_submission.csv')

In [ ]:
!pip install autokeras

In [7]:
import tensorflow as tf
import autokeras as ak

In [10]:
# '일시' 열을 날짜 데이터로 변환
data['일시'] = pd.to_datetime(data['일시'])

current_year = datetime.now().year

# 2월 29일에 해당하는 행 선택
selected_rows = data[(data['일시'].dt.month == 2) & (data['일시'].dt.day == 29)]

# 선택한 행의 인덱스를 사용하여 삭제
data = data.drop(selected_rows.index)

In [11]:
# 계절 분류 함수 정의
def classify_season(month):
    if  month == 1 or month == 2 or month == 12:
        return '겨울'
    elif 3 <= month <= 5:
        return '봄'
    elif 6 <= month <= 8:
        return '여름'
    else:
        return '가을'

# '일시' 컬럼에서 월 정보를 추출하여 '계절' 열 추가
data['월'] = data['일시'].dt.month
data['계절'] = data['월'].apply(classify_season)

# 계절별 중앙값 계산
seasonal_medians = data.groupby('계절').transform('median')

# 결측값 대체
data.fillna(seasonal_medians, inplace=True)

In [13]:
# 입력 특성과 출력 정의
x_train = data.drop(columns=['평균기온'])  # 입력 특성
y_train = data['평균기온']  # 출력 정답

# AutoML 모델 정의
regressor = ak.StructuredDataRegressor(max_trials=1)  # max_trials는 조정 가능

# 모델 훈련
regressor.fit(x_train, y_train, epochs=10)

Trial 1 Complete [00h 00m 46s]
val_loss: 22.670211791992188

Best val_loss So Far: 22.670211791992188
Total elapsed time: 00h 00m 46s
Epoch 1/10
719/719 [==============================] - 3s 3ms/step - loss: 67.6966 - mean_squared_error: 67.6966
Epoch 2/10
719/719 [==============================] - 2s 3ms/step - loss: 26.7407 - mean_squared_error: 26.7407
Epoch 3/10
719/719 [==============================] - 4s 5ms/step - loss: 23.1504 - mean_squared_error: 23.1504
Epoch 4/10
719/719 [==============================] - 2s 3ms/step - loss: 20.6675 - mean_squared_error: 20.6675
Epoch 5/10
719/719 [==============================] - 2s 3ms/step - loss: 18.4958 - mean_squared_error: 18.4958
Epoch 6/10
719/719 [==============================] - 2s 3ms/step - loss: 16.7162 - mean_squared_error: 16.7162
Epoch 7/10
719/719 [==============================] - 3s 5ms/step - loss: 15.2758 - mean_squared_error: 15.2758
Epoch 8/10
719/719 [==============================] - 2s 3ms/step - loss: 14.0473 

In [14]:
# AutoKeras 모델 확인
model = regressor.export_model()  # AutoKeras 모델을 내보내서 TensorFlow 모델로 변환
model.summary()  # TensorFlow 모델의 구조 확인

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 12)]              0         
                                                                 
 multi_category_encoding (M  (None, 12)                0         
 ultiCategoryEncoding)                                           
                                                                 
 normalization (Normalizati  (None, 12)                25        
 on)                                                             
                                                                 
 dense (Dense)               (None, 32)                416       
                                                                 
 re_lu (ReLU)                (None, 32)                0         
                                                                 
 dense_1 (Dense)             (None, 32)                1056  

In [15]:
reg_pred = regressor.predict(x_train)

submission['평균기온'] = reg_pred[:358]
print(submission.head(30))

719/719 [==============================] - 2s 2ms/step
            일시       평균기온
0   2023-01-01   0.593957
1   2023-01-02  -0.422025
2   2023-01-03  -0.853573
3   2023-01-04   3.354601
4   2023-01-05  -2.711838
5   2023-01-06  -3.076963
6   2023-01-07   4.136969
7   2023-01-08   2.871657
8   2023-01-09  -0.624630
9   2023-01-10   3.470317
10  2023-01-11   1.068488
11  2023-01-12  -0.060010
12  2023-01-13   1.552577
13  2023-01-14   1.543258
14  2023-01-15  -4.396987
15  2023-01-16  -0.572939
16  2023-01-17  -7.109331
17  2023-01-18   0.562037
18  2023-01-19   4.081431
19  2023-01-20   1.587454
20  2023-01-21  -6.314811
21  2023-01-22  -8.528751
22  2023-01-23 -11.594494
23  2023-01-24 -11.908494
24  2023-01-25  -3.066175
25  2023-01-26  -5.750393
26  2023-01-27  -9.376151
27  2023-01-28  -7.090781
28  2023-01-29  -7.055742
29  2023-01-30   0.756731


In [17]:
from sklearn.metrics import mean_absolute_error
reg_mae = mean_absolute_error(reg_pred, y_train)
print(reg_mae)

2.567952564157437
2.567952564157437


In [18]:
submission.to_csv("DEAGYUL_23_submission.csv", index=False)

print(submission.head())

           일시      평균기온
0  2023-01-01  0.593957
1  2023-01-02 -0.422025
2  2023-01-03 -0.853573
3  2023-01-04  3.354601
4  2023-01-05 -2.711838


오토ml이 답인가...